## Importing all required libraries

In [31]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

## Repeating the work done in 1st part of the assignment to build the dataframe by scraping Wikipedia page

In [106]:

from bs4 import BeautifulSoup
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
indexNames = df[ df['Borough'] == 'Not assigned'].index
# Delete these row indexes from dataFrame
df.drop(indexNames, inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [107]:
df.shape

(103, 3)

In [109]:
# Adding extra columns for longitude and latitude
df['Latitude'] = 'default value'
df['Longitude'] = 'default value'
#df.set_index('Neighborhood')
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,default value,default value
3,M4A,North York,Victoria Village,default value,default value
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",default value,default value
5,M6A,North York,"Lawrence Manor, Lawrence Heights",default value,default value
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",default value,default value
8,M9A,Etobicoke,Islington Avenue,default value,default value
9,M1B,Scarborough,"Malvern, Rouge",default value,default value
11,M3B,North York,Don Mills,default value,default value
12,M4B,East York,"Parkview Hill, Woodbine Gardens",default value,default value
13,M5B,Downtown Toronto,"Garden District, Ryerson",default value,default value


## Using Geopy API to get the longitude and latitude values for the neighborhoods in Toronto

In [110]:
for index, row in df.iterrows():
    geolocator = Nominatim(user_agent="toronto_explorer")
    address = row['Neighborhood']
    a = address + ", Toronto"
    print(index)
    print(a)
    location = geolocator.geocode(a)
    if location is None:
        print("Lets move on")
    else:
        print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
        df.loc[index,'Latitude'] = location.latitude
        df.loc[index,'Longitude'] = location.longitude

2
Parkwoods, Toronto
Latitude = 43.7587999, Longitude = -79.3201966
3
Victoria Village, Toronto
Latitude = 43.732658, Longitude = -79.3111892
4
Regent Park, Harbourfront, Toronto
Latitude = 43.64076885, Longitude = -79.37989177980148
5
Lawrence Manor, Lawrence Heights, Toronto
Latitude = 43.7152827, Longitude = -79.4439143
6
Queen's Park, Ontario Provincial Government, Toronto
Lets move on
8
Islington Avenue, Toronto
Latitude = 43.6794838, Longitude = -79.5389092
9
Malvern, Rouge, Toronto
Latitude = 43.8091955, Longitude = -79.2217008
11
Don Mills, Toronto
Latitude = 43.775347, Longitude = -79.3459439
12
Parkview Hill, Woodbine Gardens, Toronto
Lets move on
13
Garden District, Ryerson, Toronto
Lets move on
14
Glencairn, Toronto
Latitude = 43.7087117, Longitude = -79.4406853
17
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale, Toronto
Lets move on
18
Rouge Hill, Port Union, Highland Creek, Toronto
Lets move on
20
Don Mills, Toronto
Latitude = 43.775347, Longitude =

In [111]:
df_ready = df[df['Longitude']!='default value']
df_missing = df[df['Longitude']=='default value']
print(df_ready.shape)
print(df_missing.shape)

(59, 5)
(44, 5)


## I have used Geopy to get the coordinates to the neighborhoods in Toronto, but for caes where there are more than 2 neighborhoods in the same postcode, Geopy has not repturned values. So we have coordinates for 59 and missing for 44 records.

## I will now use the file given in the assignment to prepare the dataframe

In [112]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
df_downloaded = pd.read_csv('Geospatial_Coordinates.csv')
df_downloaded.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Since the loaded file is sorted by the Postal Code, in below cell I am sorting my original Dataframe df in ascending order of the postal code

In [113]:
df.sort_values(by=['Postal Code'], inplace=True)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.8092,-79.2217
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",default value,default value
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7535,-79.18
36,M1G,Scarborough,Woburn,43.7598,-79.2253
45,M1H,Scarborough,Cedarbrae,43.7565,-79.2267


## In below cell, using a for loop, I am interating through the dataframe rows, and wherever the Latitude and Longitude values are missing (did not get from Geopy), I am copying the values from the loaded dataframe

In [121]:
for index, row in df.iterrows():
    counter = 0
    postcode = row['Postal Code']
    print(index)
    print(postcode)
    if df.loc[index,'Latitude']=='default value':
        print("Have to copy data")
        counter = counter + 1
        df.loc[index,'Latitude'] = df_downloaded.loc[counter,'Latitude']
        df.loc[index,'Longitude'] = df_downloaded.loc[counter,'Longitude']
    else:
        print("Data present")        

9
M1B
Data present
18
M1C
Have to copy data
27
M1E
Data present
36
M1G
Data present
45
M1H
Data present
54
M1J
Data present
63
M1K
Have to copy data
72
M1L
Have to copy data
81
M1M
Have to copy data
90
M1N
Have to copy data
99
M1P
Have to copy data
108
M1R
Data present
117
M1S
Data present
126
M1T
Have to copy data
135
M1V
Have to copy data
144
M1W
Data present
153
M1X
Data present
46
M2H
Data present
55
M2J
Have to copy data
64
M2K
Data present
73
M2L
Data present
82
M2M
Have to copy data
91
M2N
Data present
100
M2P
Data present
109
M2R
Data present
2
M3A
Data present
11
M3B
Data present
20
M3C
Data present
47
M3H
Have to copy data
56
M3J
Have to copy data
65
M3K
Data present
74
M3L
Data present
83
M3M
Data present
92
M3N
Data present
3
M4A
Data present
12
M4B
Have to copy data
21
M4C
Data present
30
M4E
Data present
39
M4G
Data present
48
M4H
Data present
57
M4J
Data present
66
M4K
Data present
75
M4L
Data present
84
M4M
Data present
93
M4N
Data present
102
M4P
Data present
111
M4R
D

In [122]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.8092,-79.2217
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7535,-79.18
36,M1G,Scarborough,Woburn,43.7598,-79.2253
45,M1H,Scarborough,Cedarbrae,43.7565,-79.2267


In [129]:
df.sort_index(inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7588,-79.3202
3,M4A,North York,Victoria Village,43.7327,-79.3112
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6408,-79.3799
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7153,-79.4439
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.7845,-79.1605
8,M9A,Etobicoke,Islington Avenue,43.6795,-79.5389
9,M1B,Scarborough,"Malvern, Rouge",43.8092,-79.2217
11,M3B,North York,Don Mills,43.7753,-79.3459
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7845,-79.1605
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.7845,-79.1605


## The Dataframe is now ready!